## Oceanbolt Python SDK Lesson 7: Analyzing ballast legs

The shipping industry faces increasing pressure to minimize emissions. While much effort goes into understanding how to minimize emissions for the laden part of a voyage, this example will focus on understanding the ballast leg of a voyage. Due to the nature of natural endowments and trading patterns, ballasting legs are often a culprit of emissions that is ignored. We will not quantify any emissions data in this example, but purely surface insights of which routes are typical ballast legs for speficic ports.

This exercise can be done easily using the Oceanbolt Python SDK. The example here serve as Lesson 7 is  in our Python learning series. You can find past lessons on our blog: https://www.oceanbolt.com/blog/.

In [1]:
# Let's import the relevant libraries
import pandas as pd
from oceanbolt.sdk.client import APIClient
from oceanbolt.sdk.data.trade_flows import TradeFlows
from oceanbolt.sdk.distance import DistanceCalculator
from datetime import date
import numpy as np

In [2]:
# Set OB base client
base_client = APIClient("<your_token>")

In [3]:
# define parameters we will use
port_unlocode = "GNKMR" # UN/LOCODE for Santos
# include other parameters here to explore specific segments, commodity groups, regions, etc.

In [4]:
# Create tradeflow table
df_segment = TradeFlows(base_client).get(
        load_port_unlocode=[port_unlocode],
        start_date=date(2015, 1, 1),
        end_date=date(2022, 1, 25),
        max_results = 20000,
        exclude_intra_country = True,
    )

In [5]:
# create trade lane column
df_segment['ballast_lane'] = df_segment['ballast_port_name'] + ' --> ' + df_segment['load_port_name']

In [6]:
#pivot trade flow table
segment_pivot = pd.pivot_table(df_segment, values='volume', index=['ballast_lane'], aggfunc=np.count_nonzero)

In [7]:
# sort data by largest to smallest
df_sorted = segment_pivot.sort_values(by=['volume'], ascending=False)
df_sorted_renamed = df_sorted.rename({'volume': 'ballast_voyage_count'}, axis=1)
df_sorted_renamed.head(10)

,ballast_voyage_count
ballast_lane,
Yantai --> Port Kamsar,474
Jorf Lasfar --> Port Kamsar,219
Longkou --> Port Kamsar,177
San Ciprian --> Port Kamsar,98
Qingdao --> Port Kamsar,67
Taranto --> Port Kamsar,64
Rizhao --> Port Kamsar,58
Fos Sur Mer --> Port Kamsar,55
Caofeidian --> Port Kamsar,54


In [8]:
# Let us calculate the total nautical ballast miles for the Yantai --> Port Kamsar route using Oceanbolt's distance calculator
port_to_port_distance = DistanceCalculator(base_client).distance(
    locations=[
        {"unlocode": "CNYAT"},
        {"unlocode": "GNKMR"},
    ]
)

cumulative_distance = df_sorted_renamed['ballast_voyage_count'][0] * port_to_port_distance

'{0:,}'.format(round(cumulative_distance))

'4,519,948'

We therefore have that for the 474 ballast voyages from Yantai to Port Kamsar a total of 4.5 million nautical miles have been sailed in ballast since 2015.